In [1]:
!pip install transformers
!pip install sentencepiece
!pip install bert
!pip install accelerate -U
!pip install evaluate
!pip install datasets

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3744 sha256=22164d02790588996ebbaa26bc1cba15c69be55010d7de83e447b3fa09f054e2
  Stored in directory: /root/.cache/pip/wheels/d0/97/c8/5775d57c323970511488d9b53ce66cf0ded394fe4b4f6e0afb
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6779 sha256=a05dc70c4bd0511480d11806c441f2ceaf03e2abb869e8745b5fd4895d4f33b5
  Stored in directory: /root/.cache/pip/wheels/f5/ca/b1/ce71c2e0e9e0df3786697d06d53574be03ef86cd4fb0e2cb99
Successfully built bert erlastic
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFDistilBertForMultipleChoice, AdamW
import tensorflow as tf
import torch

In [5]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = TFDistilBertForMultipleChoice.from_pretrained("distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForMultipleChoice: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForMultipleChoice from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMultipleChoice from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForMultipleChoice were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a d

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [27]:
training_data = pd.read_csv('train_data.csv')

In [28]:
testing_data = pd.read_csv('test_data.csv')

In [29]:
def finding_labels():
  labels = []
  for index, item in training_data.iterrows():
    if item['label_A'] == "stereotype":
      labels.append(0)
    elif item['label_B'] == "stereotype":
      labels.append(1)
    else:
      labels.append(2)
  return labels

In [30]:
label = finding_labels()

In [31]:
first, second = [], []
sentence_type = ['sentence_A', 'sentence_B', 'sentence_C']
for index, item in training_data.iterrows():
  question = item['context'][:-1] if item['context'][-1] == '.' else item['context']
  prev_q = "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. "
  question = prev_q + "The stereotypical reason for this statement: " + question + " is "
  first_sentence = [question for i in range(3)]
  first = first + first_sentence
  second_sentence = [f"{question} {item[i]}" for i in sentence_type]
  second.append(second_sentence)

second = sum(second, [])
tokenized_examples = tokenizer(first, second, truncation=True)
element = {k: [v[i : i + 3] for i in range(0, len(v), 3)] for k, v in tokenized_examples.items()}

In [32]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import tensorflow as tf


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="tf",
        )

        batch = {k: tf.reshape(v, (batch_size, num_choices, -1)) for k, v in batch.items()}
        batch["labels"] = tf.convert_to_tensor(labels, dtype=tf.int64)
        return batch

In [33]:
import pandas as pd
from torch.utils.data import Dataset

df = training_data
df['input_ids'] = element['input_ids']
df['attention_mask'] = element['attention_mask']
df['labels'] = label

In [34]:
import datasets
from datasets import Dataset, DatasetDict

In [35]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42)

In [36]:
td_train = Dataset.from_pandas(df_train)
td_valid = Dataset.from_pandas(df_valid)

In [37]:
ds = DatasetDict()

ds['train'] = td_train
ds['validation'] = td_valid

In [38]:
import evaluate

accuracy = evaluate.load("accuracy")

In [39]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [40]:
from transformers import create_optimizer

batch_size = 12
num_train_epochs = 5
total_train_steps = (len(ds["train"]) // batch_size) * num_train_epochs
optimizer, schedule = create_optimizer(init_lr=10e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [41]:
from transformers import TFAutoModelForMultipleChoice

model = TFAutoModelForMultipleChoice.from_pretrained("distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForMultipleChoice: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForMultipleChoice from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMultipleChoice from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForMultipleChoice were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a d

In [42]:
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
tf_train_set = model.prepare_tf_dataset(
    ds["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    ds["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [43]:
model.compile(optimizer=optimizer)  # No loss argument!

In [44]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [45]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="my_distilbert_ft_model_3",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/yinani24/my_distilbert_ft_model_3 into local empty directory.


Download file tf_model.h5:   0%|          | 3.45k/256M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/256M [00:00<?, ?B/s]

In [46]:
callbacks = [metric_callback, push_to_hub_callback]

In [47]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=4, callbacks=callbacks)

Epoch 1/4
5/5 [==============================] - 255s 50s/step - loss: 1.1026 - val_loss: 1.0961 - accuracy: 0.6250
Epoch 2/4
5/5 [==============================] - 230s 49s/step - loss: 1.0855 - val_loss: 1.0854 - accuracy: 0.6250
Epoch 3/4
5/5 [==============================] - 226s 47s/step - loss: 1.0607 - val_loss: 1.0109 - accuracy: 0.6250
Epoch 4/4
5/5 [==============================] - 232s 49s/step - loss: 0.9304 - val_loss: 0.9185 - accuracy: 0.6250


In [48]:
from transformers import AutoTokenizer, TFAutoModelForMultipleChoice

In [49]:
tokenizer_new = AutoTokenizer.from_pretrained("my_distilbert_ft_model_3")
model_new = TFAutoModelForMultipleChoice.from_pretrained("my_distilbert_ft_model_3")

Some layers from the model checkpoint at my_distilbert_ft_model_3 were not used when initializing TFDistilBertForMultipleChoice: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForMultipleChoice were not initialized from the model checkpoint at my_distilbert_ft_model_3 and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
data_store = pd.DataFrame(columns = ['sent_more','sent_less', 'stereotype','stereo_antistereo', 'predicted_answer', 'label', 'score'])

In [51]:
import random

In [52]:
test_label = []
for index, item in testing_data.iterrows():
  bias_type = item['bias_type']
  question = "Which option is a stereotype?"
  # In Label, 1 means it is switched around while not in 0
  a = random.randint(0,1)
  test_label.append(0)
  options = [item['sent_more'], item['sent_less']]
  if a == 1:
    options = [item['sent_less'], item['sent_more']]
    test_label[-1] = 1

  encoded_input = tokenizer_new([[question, options[0]],
                          [question, options[1]]], return_tensors="pt", padding=True)
  inputs = {k: tf.expand_dims(v, 0) for k, v in encoded_input.items()}
  outputs = model_new(inputs)  # batch size is 1
  logits = outputs.logits

  predicted_class = tf.argmax(logits, axis=1).numpy()[0]
  predicted_answer = options[predicted_class]

  score = 0
  if predicted_answer == item['sent_more']:
    score = 1

  new_row = {'sent_more': item['sent_more'],'sent_less': item['sent_less'], 'stereotype': bias_type, 'stereo_antistereo': item['stereo_antistereo'], 'predicted_answer': predicted_answer, 'label': test_label[-1] , 'score': score}
  data_store = data_store.append(new_row, ignore_index=True)

<ipython-input-52-a96089b6f9aa>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-52-a96089b6f9aa>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-52-a96089b6f9aa>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-52-a96089b6f9aa>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-52-a96089b6f9aa>:27: FutureWarning: The frame.append method is de

In [54]:
data_store.to_csv('distilbert_stereoset_ft_crows_test_bias_positive_bias.csv',index=False)